In [1]:
import base64
import joblib
from PIL import Image
import os
import re
import time
import base64
import glob
import re
import pickle
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from scipy import interpolate
import cv2
from netCDF4 import Dataset
import matplotlib.pyplot as plt
from matplotlib.ticker import LogFormatter
os.chdir('/mnt/sdb1/wscho/Jobs/gk2aICT/src/tools/')
from api_gk2a import down_gk2a
from scipy.ndimage import gaussian_filter

import matplotlib.pyplot as plt
%matplotlib inline

from pycaret.regression import *
import joblib

In [2]:
os.chdir('/mnt/sdb1/wscho/research_etc_proj/')
from utils_common import get_train_test_data

os.chdir('/mnt/sdb1/wscho/Jobs/gk2aICT/')
from src.config import (
    LOCAL_GK2A_DIR,
    LATLON_DIR
)
from src.variables import (
    VAR2DSKEY,
    FD_VAR2FILE,
)
from src.tools.clip import (
    GK2AFDProcessor,
    GK2AEAProcessor,
    GK2AKOProcessor,

)
from src.utils import (
    read_gk2a_data
)
from src.tools.cloud_albedo import get_cloud_albedo



In [3]:
##config data 
# 준비 데이터1 : REF_TIME 및 그 전 2개 time에 해당하는 gk2a CLA 데이터(총 3개의 위성 데이터)
# 준비 데이터2 : ASOS 및 발전소 GHI 및 각각의 Metadata

REF_TIME         = '2022-05-03 12:50'        # 이용할 위성 시간대 timezone = Seoul
INTERVAL         =  15                       # 예보 시간 interval(분)
FORECAST_START   =  120                      # REF_TIME 기준 예보 시작 시간(분)
FORECAST_END     =  240                      # REF_TIME 기준 예보 종료 시간(분)


# 위성데이터 존재 directory
BASE_DIR         = '/mnt/sdb1/wscho/data_for_research/ICTgk2a/'

# 메타데이터 경로(전처리 전, 후)
META_RAW_PATH    = '/mnt/sda1/research/airflow/dags/mos_automation/data/metadata/asos_stations.pkl'
META_PATH        = '/mnt/sdb1/wscho/Jobs/gk2aICT/stn_meta.pkl'

# 구름 이동 산출물 저장 장소
GK2A_RAW_PATH    = '/mnt/sdb1/wscho/Jobs/gk2aICT/gk2a_raw.pkl'
IMG_PATH         = '/mnt/sdb1/wscho/Jobs/gk2aICT/gk2a_img.pkl'
GK2A_PATH        = '/mnt/sdb1/wscho/Jobs/gk2aICT/gk2a.pkl'

# ASOS 데이터 경로
ASOS_RAW_DIR = '/mnt/sda1/research/data/asos/mos_automation'
OBS_PATH = '/mnt/sdb1/wscho/Jobs/gk2aICT/OBS.pkl'

# Clearsky  데이터 경로
CS_PATH = '/mnt/sdb1/wscho/Jobs/gk2aICT/clear_sky_ghi.pkl'
CS_OBS_PATH = '/mnt/sdb1/wscho/Jobs/gk2aICT/clear_sky_obs_ghi.pkl'


# fitting 관련 데이터 경로
INPUT_PATH = '/mnt/sdb1/wscho/Jobs/gk2aICT/input.pkl'
INPUT_TEST_PATH = '/mnt/sdb1/wscho/Jobs/gk2aICT/input_test.pkl'
MODEL_PATH = '/mnt/sdb1/wscho/Jobs/gk2aICT/model_OD_{od}'
RESULT_PATH = '/mnt/sdb1/wscho/Jobs/gk2aICT/result_OD_{od}'



#Boundary 설정 옵션
BOUNDARY_OPTION  = True

#target 지역
KO_RANGE = {
    'ullatitude': 42.0,
    'ullongitude': 124.0,
    'lrlatitude': 33.0,
    'lrlongitude': 130.0,
}

In [4]:
#class 선언 시에 위성 resolution 지정 필요
s = time.time()
ea_gk2a = GK2AEAProcessor(resolution = 2.0)

##### PREDICT CLOUD AMOUNT ####

In [5]:
# get gk2a variable
var_set = ea_gk2a.get_gk2a_var_set('CLA', 'CA',GK2A_RAW_PATH,BASE_DIR,REF_TIME)

using data named 
'/mnt/sdb1/wscho/data_for_research/ICTgk2a/20220503/LE2/cla/gk2a_ami_le2_cla_ea020lc_202205030330.nc' 
for time '2022-05-03 12:30:00'
using data named 
'/mnt/sdb1/wscho/data_for_research/ICTgk2a/20220503/LE2/cla/gk2a_ami_le2_cla_ea020lc_202205030340.nc' 
for time '2022-05-03 12:40:00'
using data named 
'/mnt/sdb1/wscho/data_for_research/ICTgk2a/20220503/LE2/cla/gk2a_ami_le2_cla_ea020lc_202205030350.nc' 
for time '2022-05-03 12:50:00'


In [6]:
# clip to Korean Peninsula
var_clip = [ea_gk2a.cut_with_latlon(cld_x, **KO_RANGE, boundary = BOUNDARY_OPTION) for cld_x in var_set]

In [7]:
# Preprocessing before Cloud amount Prediction
for i,var in enumerate(var_clip):
    var_clip[i] = ea_gk2a.cloud_amount_preproc(var,resize=var_clip[0].shape)

In [8]:
# Select the Optical Flow method
optical_method = cv2.optflow.DualTVL1OpticalFlow_create()

In [9]:
# Calculating Cloud Motion and Save 2D Data
ea_gk2a.cal_optical_flow(optical_method,var_clip,IMG_PATH)

-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-

   First Prediction  : + 120 minutes
   Last  Prediction  : + 240 minutes

-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-

   Calculating + 15 minutes Cloud Amount

   Calculating + 30 minutes Cloud Amount

   Calculating + 45 minutes Cloud Amount

   Calculating + 60 minutes Cloud Amount

   Calculating + 75 minutes Cloud Amount

   Calculating + 90 minutes Cloud Amount

   Calculating + 105 minutes Cloud Amount

   Calculating + 120 minutes Cloud Amount

   Calculating + 135 minutes Cloud Amount

   Calculating + 150 minutes Cloud Amount

   Calculating + 165 minutes Cloud Amount

   Calculating + 180 minutes Cloud Amount

   Calculating + 195 minutes Cloud Amount

   Calculating + 210 minutes Cloud Amount

   Calculating + 225 minutes Cloud Amount

   Calculating + 240 minutes Cloud Amount



In [10]:
# Pick target grid data with metadata
ea_gk2a.pick_var_target(GK2A_PATH,IMG_PATH,META_PATH, REF_TIME)

대관령 y,x : 232, 197
춘천 y,x : 221, 155
백령도 y,x : 217, 25
북강릉 y,x : 225, 203
강릉 y,x : 228, 205
서울 y,x : 239, 122
인천 y,x : 245, 107
원주 y,x : 251, 165
울릉도 y,x : 237, 264
수원 y,x : 256, 123
충주 y,x : 271, 165
서산 y,x : 283, 102
청주 y,x : 290, 143
대전 y,x : 304, 141
추풍령 y,x : 312, 168
안동 y,x : 292, 199
상주 y,x : 301, 175
포항 y,x : 320, 230
대구 y,x : 330, 198
전주 y,x : 333, 130
창원 y,x : 368, 195
광주 y,x : 370, 120
부산 y,x : 371, 216
목포 y,x : 389, 98
여수 y,x : 393, 159
흑산도 y,x : 396, 56
고창 y,x : 360, 107
홍성 y,x : 289, 111
제주 y,x : 460, 105
고산 y,x : 472, 88
진주 y,x : 369, 172
제천 y,x : 261, 176
고창군 y,x : 356, 112
영광군 y,x : 364, 102
김해시 y,x : 364, 210
순창군 y,x : 359, 131
북창원 y,x : 365, 200
양산시 y,x : 360, 215
보성군 y,x : 392, 135
강진군 y,x : 399, 115
의령군 y,x : 360, 183
함양군 y,x : 351, 158
광양시 y,x : 382, 156
청송군 y,x : 299, 214
경주시 y,x : 332, 222
북춘천 y,x : 219, 156
철원 y,x : 208, 136
동두천 y,x : 222, 126
파주 y,x : 223, 113
대관령 y,x : 232, 197
춘천 y,x : 221, 155
백령도 y,x : 217, 25
북강릉 y,x : 225, 203
강릉 y,x : 228, 205
서울 y,x : 

##### MAKE MODEL WITH ASOS OBS ####

In [11]:
# Get asos_meta & ASOS RAW data
asos_meta = pd.read_pickle(META_RAW_PATH)

In [12]:
asos_list   =  glob.glob(os.path.join(ASOS_RAW_DIR, 'asos_db_202[0-2]*.pkl'))
asos_list.sort()

raw_asos_data = pd.DataFrame()
for data in asos_list:
    asos_tmp = pd.read_pickle(data)
    raw_asos_data = raw_asos_data.append(asos_tmp)

In [13]:
asos_data = raw_asos_data.groupby('stn_nm').apply(lambda x : x.drop(x.index) if all(x.ghi.isna()) else x).rename(columns={ 
                                                                'timestamp' : 'dt',
                                                                'ghi'       : 'true_ghi',}).reset_index(drop=True)
asos_data = pd.merge(asos_data,asos_meta[['stn_id','latitude','longitude','altitude']],on='stn_id').set_index('dt')
asos_data['stn_id'] = asos_data.stn_id.astype("int").astype("string")
asos_data.to_pickle(OBS_PATH)

In [14]:
# Calculating Clear Sky Radiation
ea_gk2a.cal_clear_sky(CS_PATH, META_PATH,OBS_PATH,method='OBS')

# Get Corrected ASOS GHI
ea_gk2a.preproc_asos_ghi(CS_OBS_PATH,CS_PATH,META_PATH)

대관령
춘천
백령도
북강릉
강릉
서울
인천
원주
울릉도
수원
충주
서산
청주
대전
추풍령
안동
상주
포항
대구
전주
창원
광주
부산
목포
여수
흑산도
고창
홍성
제주
고산
진주
제천
고창군
영광군
김해시
순창군
북창원
양산시
보성군
강진군
의령군
함양군
광양시
청송군
경주시
북춘천
철원
동두천
파주
.
.
.
If you did not use ASOS data, never use this function
.
.
.

대관령
춘천
백령도
북강릉
강릉
서울
인천
원주
울릉도
수원
충주
서산
청주
대전
추풍령
안동
상주
포항
대구
전주
창원
광주
부산
목포
여수
흑산도
고창
홍성
제주
고산
진주
제천
고창군
영광군
김해시
순창군
북창원
양산시
보성군
강진군
의령군
함양군
광양시
청송군
경주시
북춘천
철원
동두천
파주


In [15]:
# Prepare features(Common)
ea_gk2a.prepare_features(INPUT_PATH,CS_OBS_PATH,method ='OBS')

In [16]:
# Prepare features(Specific)
ghi_test_set = []
ghi_train_set = []
trs_switch = True

input_data =pd.read_pickle(INPUT_PATH)

for z,cur_cloud_OD in enumerate(np.arange(0,1.01,0.1)):

    cur_cloud_OD +=0.05
    cur_cloud_OD = cur_cloud_OD.clip(0,1)

    if trs_switch :  trs  =  0.025
    else      :  trs  =  100000

    tmp_ghi = input_data[input_data.zenith<90].copy()
    tmp_ghi = tmp_ghi[abs(tmp_ghi.cloud_index - cur_cloud_OD)<trs]
    tmp_ghi.cal_Cloud_OD = tmp_ghi.cal_Cloud_OD.round(4)
    tmp_ghi = tmp_ghi.dropna()

    ghi_train, ghi_test ,__ ,__ = get_train_test_data(tmp_ghi, start_date='2020-01-01',
                                                   end_date='2022-12-31', split_freq=5)
    ghi_test_set.append(ghi_test)
    ghi_train_set.append(ghi_train)

In [17]:
# Fitting
ea_gk2a.fitting_ghi_ref(MODEL_PATH,RESULT_PATH, ghi_train_set)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,0.0878,0.0140,0.1184,0.2350,0.0964,5.0077
1,0.0869,0.0135,0.1163,0.2134,0.0956,8.1137
2,0.0884,0.0145,0.1205,0.2686,0.0970,6.0365
3,0.0879,0.0138,0.1176,0.2487,0.0962,9.0002
4,0.0858,0.0137,0.1172,0.2531,0.0950,6.9068
5,0.0868,0.0140,0.1185,0.2538,0.0957,8.8052
6,0.0869,0.0136,0.1167,0.2427,0.0957,7.9295
7,0.0879,0.0142,0.1193,0.2147,0.0970,4.1938
8,0.0862,0.0138,0.1174,0.2428,0.0953,5.9967
9,0.0873,0.0138,0.1176,0.2419,0.0957,5.7092


Transformation Pipeline and Model Succesfully Saved
0.08554800917394711


#### PREDICT GHI ###

In [11]:
# Cal. cs Rad. for predict time
ea_gk2a.cal_clear_sky(CS_PATH, META_PATH,GK2A_PATH,method='gk2a')

대관령
춘천
백령도
북강릉
강릉
서울
인천
원주
울릉도
수원
충주
서산
청주
대전
추풍령
안동
상주
포항
대구
전주
창원
광주
부산
목포
여수
흑산도
고창
홍성
제주
고산
진주
제천
고창군
영광군
김해시
순창군
북창원
양산시
보성군
강진군
의령군
함양군
광양시
청송군
경주시
북춘천
철원
동두천
파주


In [12]:
# Prepare Features(common)
ea_gk2a.prepare_features(INPUT_PATH,CS_PATH,method='gk2a')

In [13]:
# Prpare features(specific)
trs_switch = True

input_data =pd.read_pickle(INPUT_PATH)

input_data = input_data[input_data.zenith<90].copy()
input_data.cal_Cloud_OD = input_data.cal_Cloud_OD.round(4)
input_data = input_data.dropna()
input_data['OD_REF'] = input_data.cloud_index.round(1)

In [14]:
# Preduct GHI
model_col = ['azimuth', 'zenith','cs_ghi','cs_dni', 'cs_dhi', 'cloud_index','latitude','longitude', 'altitude', 'cal_Cloud_OD']
result_data = pd.DataFrame()
for z in range(10):
    cur_input = input_data[input_data.OD_REF == z]
    model = load_model(MODEL_PATH.format(od = z))
    try:
        prediction = predict_model(model, data=cur_input[model_col])
        result_data = result_data.append(prediction)
    except:
        print(f'There is no cloud_index == {0.1*z}')

Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
There is no cloud_index == 0.1
Transformation Pipeline and Model Successfully Loaded
There is no cloud_index == 0.2
Transformation Pipeline and Model Successfully Loaded
There is no cloud_index == 0.30000000000000004
Transformation Pipeline and Model Successfully Loaded
There is no cloud_index == 0.4
Transformation Pipeline and Model Successfully Loaded
There is no cloud_index == 0.5
Transformation Pipeline and Model Successfully Loaded
There is no cloud_index == 0.6000000000000001
Transformation Pipeline and Model Successfully Loaded
There is no cloud_index == 0.7000000000000001
Transformation Pipeline and Model Successfully Loaded
There is no cloud_index == 0.8
Transformation Pipeline and Model Successfully Loaded
There is no cloud_index == 0.9


In [15]:
#Postprocessing
meta_data = pd.read_pickle(META_PATH)
result_data = result_data.rename(columns={'Label' : 'predict_OD'})
result_data['predict_ghi'] = (result_data.predict_OD * result_data.cs_dni + result_data.cs_dhi)
result_data = result_data.reset_index().merge(meta_data[['stn_id','stn_nm','latitude','longitude']],on=['latitude','longitude']).set_index('dt')
result_data = result_data.sort_values(by=['dt','stn_id'])
col1=result_data.columns[-2:].to_list()
col2=result_data.columns[:-2].to_list()
new_col=col1+col2
result_data=result_data[new_col]

In [16]:
#Result!!
result_data

,stn_id,stn_nm,azimuth,zenith,cs_ghi,cs_dni,cs_dhi,cloud_index,latitude,longitude,altitude,cal_Cloud_OD,predict_OD,predict_ghi
dt,,,,,,,,,,,,,,
2022-05-03 14:50:00+09:00,127,충주,246.174463,38.458940,762.167913,820.339703,119.680201,0.05,36.9705,127.9525,114.85,0.7742,0.799152,775.256555
2022-05-03 14:50:00+09:00,129,서산,244.905083,37.316521,797.636662,876.189946,100.685394,0.05,36.7766,126.4939,25.25,0.7868,0.767379,773.054778
2022-05-03 14:50:00+09:00,131,청주,246.041848,37.950069,772.959360,832.453366,116.414361,0.05,36.6392,127.4407,58.70,0.7799,0.799117,781.642111
2022-05-03 14:50:00+09:00,133,대전,246.286688,37.791646,777.327668,839.107396,114.111667,0.05,36.3720,127.3721,70.22,0.7816,0.809177,793.098030
2022-05-03 14:50:00+09:00,135,추풍령,247.087786,38.192382,770.585675,832.429568,116.231248,0.05,36.2203,127.9946,244.98,0.7772,0.811758,791.962652
2022-05-03 14:50:00+09:00,136,안동,247.381134,38.857826,752.110415,806.648193,123.851582,0.05,36.5729,128.7073,141.26,0.7697,0.810513,777.650452
2022-05-03 14:50:00+09:00,137,상주,247.029540,38.386513,761.639012,816.934157,121.176981,0.05,36.4084,128.1574,96.58,0.7750,0.809379,782.386015
2022-05-03 14:50:00+09:00,138,포항,248.649236,39.158069,750.111903,807.600158,123.771361,0.05,36.0320,129.3800,3.94,0.7663,0.778150,752.205042
2022-05-03 14:50:00+09:00,143,대구,248.131064,38.554208,757.052222,809.577677,123.829911,0.05,35.8780,128.6530,54.27,0.7731,0.791945,764.970811


In [17]:
e = time.time()
print(e-s)

27.864564657211304


In [ ]:
120 ~ 240 분 예측 데이터 내놓는데 30초 걸립니다